# Chatbot with Message Summarization - Practice Exercises

## Overview
This notebook provides hands-on exercises to practice building conversational AI systems with intelligent message summarization. You'll learn to create chatbots that maintain compressed conversation history while preserving important context.

## Learning Objectives
By the end of these exercises, you will:
- Build chatbots with custom state that includes conversation summaries
- Implement intelligent summarization strategies for conversation history
- Use conditional logic to determine when summarization should occur
- Manage conversation memory efficiently with LangGraph checkpointers
- Handle long-running conversations without losing important context
- Create different summarization approaches for various use cases

## Prerequisites
- Completed the chatbot-summarization.ipynb tutorial
- Understanding of MessagesState and message management
- Knowledge of LangGraph conditional edges and checkpointers

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langchain_core langgraph langchain_openai

In [45]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

# Optional: Set up LangSmith for tracing
_set_env("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langchain-academy"

## Exercise 1: Basic Summarization Chatbot

### Task
Create a chatbot that maintains a running summary of the conversation and uses it to provide context for responses.

### TODO: Set up basic summarization system

In [46]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage, AIMessage
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from IPython.display import Image, display
from typing import Literal

# TODO: Initialize the chat model
model = ChatOpenAI(model="gpt-5-nano", temperature=0)

# TODO: Define state with summary
class SummarizationState(MessagesState):
    # TODO: Add summary field as string
    summary:str 

In [47]:
# TODO: Implement chat node with summary integration
def chat_with_summary(state: SummarizationState):
    print("Generating response with conversation context...")
    
    # TODO: Get the current summary
    summary = state.get("summary", "")
    messages = state["messages"]
    
    # TODO: If there's a summary, add it as a system message
    if summary:
        message2send = [SystemMessage(content=f'Summary of conversion so far:\n\n{summary}')] + messages
    else:
        message2send = messages
    
    # TODO: Get response from model and return it
    #need to return a dictionary
    return {'messages':model.invoke(message2send)}

# TODO: Implement basic summarization node
def create_summary(state: SummarizationState):
    print("Creating conversation summary...")
    
    messages = state["messages"]
    current_summary = state.get("summary", "")
    
    # TODO: Create summarization prompt
    if current_summary:
        # TODO: Create prompt to extend existing summary
        summary_prompt = f"Previous summary: {current_summary}\n\nExtend this summary with the new messages above:"
    else:
        # TODO: Create prompt to create initial summary
        summary_prompt = "Create a concise summary of the conversation above:"
    
    # TODO: Add summary prompt to messages and get summary from model
    # TODO: Remove old messages except the last 2 using RemoveMessage
    # TODO: Return updated summary and trimmed messages
    results = model.invoke(messages+[HumanMessage(content=summary_prompt)])
    message2remove = [RemoveMessage(id=msg.id) for msg in messages[:-2]]
    return {'messages':message2remove+[results]}


# TODO: Implement decision function for when to summarize
def should_summarize(state: SummarizationState) -> Literal["create_summary", END]:
    messages = state["messages"]
    # TODO: Return "summarize" if more than 6 messages, otherwise END
    if len(messages) >6:
        return 'create_summary'
    else:
        return END

In [ ]:
create_summary({'messages':[HumanMessage('hi')]})

Creating conversation summary...


In [ ]:
# TODO: Build summarization chatbot graph
builder_basic_summary = StateGraph(SummarizationState)
# TODO: Add nodes: chat_with_summary, create_summary
builder_basic_summary.add_node('chat_with_summary', chat_with_summary)
builder_basic_summary.add_node('create_summary', create_summary)
# TODO: Add edges: START -> chat_with_summary
builder_basic_summary.add_edge(START, 'chat_with_summary')
# TODO: Add conditional edges: chat_with_summary -> (should_summarize) -> summarize/continue
builder_basic_summary.add_conditional_edges('chat_with_summary', should_summarize)
# TODO: Add edges: create_summary -> END, and "continue" -> END
builder_basic_summary.add_edge('create_summary', END)
# Add memory
memory = MemorySaver()
graph_basic_summary = builder_basic_summary.compile(checkpointer=memory)
# display(Image(graph_basic_summary.get_graph().draw_mermaid_png()))

In [ ]:
# TODO: Test basic summarization chatbot
config = {"configurable": {"thread_id": "basic_summary_test"}}

# Start conversation with multiple turns to trigger summarization
conversation_turns = [
    "Hi, I'm working on a Python project and need some help.",
    "I'm building a web scraper using BeautifulSoup.",
    "I'm having trouble parsing nested HTML elements.",
    "Can you show me how to extract data from tables?",
    "What about handling JavaScript-rendered content?",
    "Should I use Selenium for that?",
    "How do I handle rate limiting when scraping?",
    "What's the best practice for storing scraped data?"
]

for i, user_input in enumerate(conversation_turns, 1):
    print(f"\n=== Turn {i} ===")
    print(f"User: {user_input}")
    
    # Add user message and invoke
    result = graph_basic_summary.invoke(
        {"messages": [HumanMessage(content=user_input)]}, 
        config
    )
    
    # Print AI response
    last_message = result["messages"][-1]
    print(f"Assistant: {last_message.content[:100]}...")
    
    # Print summary if it exists
    if result.get("summary"):
        print(f"\nSummary: {result['summary'][:150]}...")
    
    print(f"Total messages in state: {len(result['messages'])}")

## Exercise 2: Advanced Summarization Strategies

### Task
Implement different summarization strategies based on conversation type and user preferences.

### TODO: Create advanced summarization system

In [ ]:
from typing import Dict, Any, List
from enum import Enum

# TODO: Define summarization strategies
class SummaryStrategy(Enum):
    CHRONOLOGICAL = "chronological"  # Timeline-based summary
    TOPICAL = "topical"             # Topic-based summary
    HIERARCHICAL = "hierarchical"   # Main points with sub-points
    CONVERSATIONAL = "conversational" # Dialogue-focused summary

# TODO: Define advanced state
class AdvancedSummarizationState(MessagesState):
    summary: str
    # TODO: Add summary_strategy, conversation_topics, summary_length
    pass

In [ ]:
# TODO: Implement strategy-specific summarization
def create_strategic_summary(state: AdvancedSummarizationState):
    print(f"Creating {state['summary_strategy'].value} summary...")
    
    messages = state["messages"]
    current_summary = state.get("summary", "")
    strategy = state["summary_strategy"]
    length = state.get("summary_length", "medium")
    
    # TODO: Create strategy-specific prompts
    strategy_prompts = {
        SummaryStrategy.CHRONOLOGICAL: "Create a chronological summary showing the progression of the conversation:",
        SummaryStrategy.TOPICAL: "Create a topical summary organizing information by main subjects discussed:",
        SummaryStrategy.HIERARCHICAL: "Create a hierarchical summary with main points and supporting details:",
        SummaryStrategy.CONVERSATIONAL: "Create a conversational summary focusing on the dialogue flow and context:"
    }
    
    # TODO: Add length specifications
    length_specs = {
        "brief": " Keep it under 50 words.",
        "medium": " Keep it under 150 words.", 
        "detailed": " Provide comprehensive details up to 300 words."
    }
    
    # TODO: Combine strategy and length into final prompt
    # TODO: Get summary from model and clean up old messages
    pass

# TODO: Implement topic detection for dynamic strategy selection
def detect_conversation_topics(state: AdvancedSummarizationState):
    print("Detecting conversation topics...")
    
    messages = state["messages"]
    # TODO: Analyze recent messages to identify main topics
    # TODO: Use simple keyword-based detection or LLM-based analysis
    # TODO: Update conversation_topics in state
    
    # Example topic categories
    topic_keywords = {
        "technical": ["code", "programming", "API", "database", "algorithm"],
        "business": ["strategy", "revenue", "customer", "market", "sales"],
        "creative": ["design", "art", "creative", "visual", "aesthetic"],
        "personal": ["feel", "think", "opinion", "experience", "personal"]
    }
    
    # TODO: Implement topic detection and return updated state
    pass

# TODO: Implement adaptive strategy selector
def select_summary_strategy(state: AdvancedSummarizationState):
    print("Selecting optimal summarization strategy...")
    
    topics = state.get("conversation_topics", [])
    message_count = len(state["messages"])
    
    # TODO: Select strategy based on:
    # - Conversation topics (technical -> hierarchical, personal -> conversational)
    # - Message count (few messages -> chronological, many -> topical)
    # - Existing summary length
    
    # Simple selection logic (enhance this)
    if "technical" in topics:
        selected_strategy = SummaryStrategy.HIERARCHICAL
    elif message_count > 10:
        selected_strategy = SummaryStrategy.TOPICAL
    else:
        selected_strategy = SummaryStrategy.CHRONOLOGICAL
    
    # TODO: Return updated state with selected strategy
    pass

In [ ]:
# TODO: Build advanced summarization graph
builder_advanced = StateGraph(AdvancedSummarizationState)
# TODO: Add nodes: chat_with_summary, detect_conversation_topics, 
#       select_summary_strategy, create_strategic_summary
# TODO: Create flow: chat -> detect_topics -> select_strategy -> create_summary
# TODO: Add conditional logic for when to summarize

graph_advanced = builder_advanced.compile(checkpointer=MemorySaver())
display(Image(graph_advanced.get_graph().draw_mermaid_png()))

In [ ]:
# TODO: Test advanced summarization with different conversation types
test_scenarios = [
    {
        "name": "Technical Discussion",
        "messages": [
            "I need help with database optimization.",
            "My queries are running slowly on large datasets.",
            "I'm using PostgreSQL with a table of 10 million rows.",
            "What indexing strategies would you recommend?",
            "Should I consider partitioning the table?",
            "What about query caching mechanisms?"
        ],
        "expected_strategy": SummaryStrategy.HIERARCHICAL
    },
    {
        "name": "Creative Project",
        "messages": [
            "I'm working on a logo design for a tech startup.",
            "The client wants something modern and minimalist.",
            "I'm thinking of using blue and gray colors.",
            "What fonts would work well for a tech company?",
            "Should the logo work well in both light and dark themes?"
        ],
        "expected_strategy": SummaryStrategy.CONVERSATIONAL
    }
]

for scenario in test_scenarios:
    print(f"\n=== Testing: {scenario['name']} ===")
    config = {"configurable": {"thread_id": f"advanced_{scenario['name'].lower()}"}}
    
    # Initialize with default state
    initial_state = {
        "summary": "",
        "summary_strategy": SummaryStrategy.CHRONOLOGICAL,
        "conversation_topics": [],
        "summary_length": "medium"
    }
    
    # Process each message
    for message in scenario["messages"]:
        result = graph_advanced.invoke(
            {**initial_state, "messages": [HumanMessage(content=message)]},
            config
        )
        initial_state = result  # Update for next iteration
    
    print(f"Selected strategy: {result['summary_strategy'].value}")
    print(f"Topics detected: {result['conversation_topics']}")
    if result.get("summary"):
        print(f"Summary: {result['summary'][:200]}...")

## Exercise 3: Multi-Level Summarization System

### Task
Create a sophisticated summarization system with multiple levels of detail and the ability to maintain both short-term and long-term conversation memory.

### TODO: Implement hierarchical summarization

In [ ]:
from datetime import datetime
from typing import Optional

# TODO: Define multi-level state
class MultiLevelSummaryState(MessagesState):
    # TODO: Add different summary levels
    # short_term_summary: str (last few exchanges)
    # medium_term_summary: str (recent topics)
    # long_term_summary: str (overall conversation themes)
    # summary_metadata: Dict[str, Any] (timestamps, token counts, etc.)
    pass

# TODO: Implement hierarchical summarization node
def create_hierarchical_summary(state: MultiLevelSummaryState):
    print("Creating multi-level summaries...")
    
    messages = state["messages"]
    current_short = state.get("short_term_summary", "")
    current_medium = state.get("medium_term_summary", "")
    current_long = state.get("long_term_summary", "")
    
    # TODO: Create short-term summary (last 3-4 exchanges)
    recent_messages = messages[-6:]  # Last 6 messages
    # TODO: Generate short-term summary
    
    # TODO: Update medium-term summary (incorporate short-term into broader context)
    # TODO: Update long-term summary (main themes and patterns)
    
    # TODO: Create metadata
    metadata = {
        "last_updated": datetime.now().isoformat(),
        "total_exchanges": len(messages) // 2,
        "summary_version": "1.0"
    }
    
    # TODO: Return updated summaries and clean up old messages
    pass

# TODO: Implement context-aware response generation
def generate_contextual_response(state: MultiLevelSummaryState):
    print("Generating response with multi-level context...")
    
    messages = state["messages"]
    short_summary = state.get("short_term_summary", "")
    medium_summary = state.get("medium_term_summary", "")
    long_summary = state.get("long_term_summary", "")
    
    # TODO: Build layered context prompt
    context_layers = []
    if long_summary:
        context_layers.append(f"Overall conversation context: {long_summary}")
    if medium_summary:
        context_layers.append(f"Recent topics: {medium_summary}")
    if short_summary:
        context_layers.append(f"Immediate context: {short_summary}")
    
    # TODO: Create system message with layered context
    # TODO: Generate response and return it
    pass

# TODO: Implement intelligent summarization trigger
def should_create_multilevel_summary(state: MultiLevelSummaryState) -> Literal["summarize", "continue"]:
    messages = state["messages"]
    metadata = state.get("summary_metadata", {})
    
    # TODO: More sophisticated triggering logic:
    # - Message count thresholds for different summary levels
    # - Time-based triggers
    # - Topic shift detection
    
    # Simple implementation
    if len(messages) > 8:
        return "summarize"
    return "continue"

In [ ]:
# TODO: Build multi-level summarization graph
builder_multilevel = StateGraph(MultiLevelSummaryState)
# TODO: Add nodes and edges for hierarchical processing

graph_multilevel = builder_multilevel.compile(checkpointer=MemorySaver())
display(Image(graph_multilevel.get_graph().draw_mermaid_png()))

In [ ]:
# TODO: Test multi-level summarization
config_multilevel = {"configurable": {"thread_id": "multilevel_test"}}

# Simulate a long, complex conversation
complex_conversation = [
    "I'm planning a career transition from finance to tech.",
    "I have a background in financial modeling and data analysis.",
    "What programming languages should I focus on learning first?",
    "I'm particularly interested in data science and machine learning.",
    "Should I get a formal degree or focus on online courses and bootcamps?",
    "How important are certifications in the tech industry?",
    "I'm also considering freelancing while I build up my skills.",
    "What types of projects would be good for a portfolio?",
    "How do I network effectively in the tech community?",
    "What salary range should I expect for entry-level positions?"
]

for i, user_input in enumerate(complex_conversation, 1):
    print(f"\n=== Exchange {i} ===")
    print(f"User: {user_input}")
    
    result = graph_multilevel.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config_multilevel
    )
    
    # Print response
    last_message = result["messages"][-1]
    print(f"Assistant: {last_message.content[:100]}...")
    
    # Print summaries if they exist
    if result.get("short_term_summary"):
        print(f"\nShort-term: {result['short_term_summary'][:80]}...")
    if result.get("medium_term_summary"):
        print(f"Medium-term: {result['medium_term_summary'][:80]}...")
    if result.get("long_term_summary"):
        print(f"Long-term: {result['long_term_summary'][:80]}...")
    
    if result.get("summary_metadata"):
        metadata = result["summary_metadata"]
        print(f"\nMetadata: {metadata.get('total_exchanges', 0)} exchanges, last updated: {metadata.get('last_updated', 'N/A')[:19]}")

## Exercise 4: Summarization with External Knowledge Integration

### Task
Create a chatbot that not only summarizes conversations but also integrates relevant external knowledge into the summaries for enhanced context.

### TODO: Implement knowledge-enhanced summarization

In [ ]:
# TODO: Define knowledge-enhanced state
class KnowledgeEnhancedState(MessagesState):
    summary: str
    # TODO: Add external_knowledge, knowledge_sources, enhanced_context
    pass

# TODO: Implement knowledge retrieval (simulated)
def retrieve_relevant_knowledge(topics: List[str]) -> Dict[str, str]:
    """Simulate retrieving relevant knowledge based on conversation topics."""
    # TODO: In a real system, this would query external knowledge bases
    knowledge_db = {
        "python": "Python is a high-level programming language known for its simplicity and versatility in data science, web development, and automation.",
        "machine_learning": "Machine learning is a subset of AI that enables computers to learn and make decisions from data without explicit programming.",
        "career_transition": "Career transitions in tech often benefit from building a portfolio, networking, and continuous learning of relevant skills.",
        "database": "Databases are structured collections of data that can be efficiently stored, retrieved, and managed using database management systems."
    }
    
    # TODO: Return relevant knowledge snippets
    relevant_knowledge = {}
    for topic in topics:
        for key, value in knowledge_db.items():
            if key.lower() in topic.lower() or topic.lower() in key.lower():
                relevant_knowledge[key] = value
    
    return relevant_knowledge

# TODO: Implement knowledge-enhanced summarization
def create_knowledge_enhanced_summary(state: KnowledgeEnhancedState):
    print("Creating knowledge-enhanced summary...")
    
    messages = state["messages"]
    current_summary = state.get("summary", "")
    
    # TODO: Extract topics from recent conversation
    # Simple keyword extraction (enhance with NLP in real implementation)
    message_text = " ".join([msg.content for msg in messages[-6:] if hasattr(msg, 'content')])
    
    # TODO: Retrieve relevant external knowledge
    topics = ["python", "machine_learning", "career", "database"]  # Simplified topic detection
    external_knowledge = retrieve_relevant_knowledge(topics)
    
    # TODO: Create enhanced summary prompt that incorporates external knowledge
    knowledge_context = "\n".join([f"{k}: {v}" for k, v in external_knowledge.items()])
    
    # TODO: Generate enhanced summary using both conversation and knowledge
    # TODO: Return updated state with enhanced summary and knowledge sources
    pass

# TODO: Implement contextual response with knowledge
def respond_with_knowledge_context(state: KnowledgeEnhancedState):
    print("Generating response with knowledge context...")
    
    messages = state["messages"]
    summary = state.get("summary", "")
    external_knowledge = state.get("external_knowledge", {})
    
    # TODO: Build comprehensive context including summary and external knowledge
    # TODO: Generate response and return it
    pass

In [ ]:
# TODO: Build knowledge-enhanced graph
builder_knowledge = StateGraph(KnowledgeEnhancedState)
# TODO: Add nodes and create flow with knowledge integration

graph_knowledge = builder_knowledge.compile(checkpointer=MemorySaver())
display(Image(graph_knowledge.get_graph().draw_mermaid_png()))

In [ ]:
# TODO: Test knowledge-enhanced summarization
config_knowledge = {"configurable": {"thread_id": "knowledge_enhanced"}}

knowledge_conversation = [
    "I'm learning Python for data analysis.",
    "I want to understand machine learning algorithms better.",
    "What's the difference between supervised and unsupervised learning?",
    "Can you recommend some good datasets for practice?",
    "I'm also interested in database management for storing my analysis results."
]

for i, user_input in enumerate(knowledge_conversation, 1):
    print(f"\n=== Knowledge-Enhanced Exchange {i} ===")
    print(f"User: {user_input}")
    
    result = graph_knowledge.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config_knowledge
    )
    
    # Print response
    last_message = result["messages"][-1]
    print(f"Assistant: {last_message.content[:150]}...")
    
    # Print enhanced summary and knowledge
    if result.get("summary"):
        print(f"\nEnhanced Summary: {result['summary'][:200]}...")
    
    if result.get("external_knowledge"):
        print(f"\nExternal Knowledge Used: {list(result['external_knowledge'].keys())}")

## Exercise 5: Adaptive Summarization with User Preferences

### Task
Create a personalized summarization system that adapts to user preferences and conversation patterns over time.

### TODO: Implement adaptive summarization

In [ ]:
# TODO: Define adaptive state with user preferences
class AdaptiveSummaryState(MessagesState):
    summary: str
    # TODO: Add user_preferences, conversation_patterns, adaptation_history
    pass

# TODO: Implement user preference learning
def learn_user_preferences(state: AdaptiveSummaryState):
    print("Learning user preferences...")
    
    messages = state["messages"]
    current_prefs = state.get("user_preferences", {})
    
    # TODO: Analyze user behavior patterns:
    # - Preferred response length (based on user message lengths)
    # - Topic preferences (based on frequency)
    # - Communication style (formal vs casual)
    # - Question types (how-to, what-is, why, etc.)
    
    # Simple preference detection (enhance with ML in real implementation)
    user_messages = [msg for msg in messages if isinstance(msg, HumanMessage)]
    
    # TODO: Calculate and update preferences
    preferences = {
        "avg_message_length": sum(len(msg.content) for msg in user_messages[-5:]) / min(5, len(user_messages)),
        "prefers_detailed_responses": "explain" in " ".join([msg.content.lower() for msg in user_messages[-3:]]),
        "asks_followup_questions": len([msg for msg in user_messages[-3:] if "?" in msg.content]) > 1
    }
    
    # TODO: Return updated preferences
    pass

# TODO: Implement adaptive summarization based on preferences
def create_adaptive_summary(state: AdaptiveSummaryState):
    print("Creating adaptive summary based on user preferences...")
    
    messages = state["messages"]
    current_summary = state.get("summary", "")
    preferences = state.get("user_preferences", {})
    
    # TODO: Adapt summarization based on learned preferences:
    # - Adjust summary length based on user's typical message length
    # - Include more technical details if user prefers detailed responses
    # - Focus on actionable items if user asks many follow-up questions
    
    # TODO: Create personalized summary prompt
    # TODO: Generate and return adaptive summary
    pass

# TODO: Implement preference-aware response generation
def generate_preference_aware_response(state: AdaptiveSummaryState):
    print("Generating preference-aware response...")
    
    messages = state["messages"]
    summary = state.get("summary", "")
    preferences = state.get("user_preferences", {})
    
    # TODO: Customize response style based on preferences
    # TODO: Generate and return personalized response
    pass

In [ ]:
# TODO: Build adaptive summarization graph
builder_adaptive = StateGraph(AdaptiveSummaryState)
# TODO: Add nodes for preference learning and adaptive processing

graph_adaptive = builder_adaptive.compile(checkpointer=MemorySaver())
display(Image(graph_adaptive.get_graph().draw_mermaid_png()))

In [ ]:
# TODO: Test adaptive summarization
config_adaptive = {"configurable": {"thread_id": "adaptive_test"}}

# Simulate user with specific communication patterns
adaptive_conversation = [
    "Can you explain how neural networks work? I want to understand the basic concepts.",
    "That's helpful! Can you go deeper into backpropagation? How exactly does the learning happen?",
    "I see. What about different types of neural networks? Are there variations for different tasks?",
    "Interesting! How do I choose the right architecture for my specific problem? What factors should I consider?",
    "Great explanation! Can you recommend some resources where I can practice building these networks?"
]

for i, user_input in enumerate(adaptive_conversation, 1):
    print(f"\n=== Adaptive Exchange {i} ===")
    print(f"User: {user_input}")
    
    result = graph_adaptive.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config_adaptive
    )
    
    # Print response
    last_message = result["messages"][-1]
    print(f"Assistant: {last_message.content[:150]}...")
    
    # Print learned preferences and adaptive summary
    if result.get("user_preferences"):
        prefs = result["user_preferences"]
        print(f"\nLearned Preferences: {prefs}")
    
    if result.get("summary"):
        print(f"\nAdaptive Summary: {result['summary'][:200]}...")

## Challenge Exercise: Multi-User Conversation Summarization

### Task
Create a system that can handle multi-user conversations (like group chats) and create personalized summaries for each participant based on their involvement and interests.

### TODO: Implement multi-user summarization system

In [ ]:
# TODO: Define multi-user conversation state
class MultiUserConversationState(MessagesState):
    # TODO: Add participant_summaries, participant_profiles, conversation_threads
    pass

# TODO: Implement participant tracking
def track_participants(state: MultiUserConversationState):
    """Track which participants are involved and their conversation patterns."""
    # TODO: Identify different speakers/users in the conversation
    # TODO: Track their topics of interest and participation patterns
    # TODO: Update participant profiles
    pass

# TODO: Implement personalized summarization for each participant
def create_personalized_summaries(state: MultiUserConversationState):
    """Create personalized summaries for each conversation participant."""
    # TODO: Generate different summaries based on:
    # - What each participant contributed
    # - What they seemed most interested in
    # - Action items relevant to them
    # - Topics they asked about
    pass

print("Multi-user conversation system defined - implement participant tracking and personalized summarization!")

## Summary

In these exercises, you've practiced:
- Building chatbots with conversation summarization capabilities
- Implementing different summarization strategies (chronological, topical, hierarchical, conversational)
- Creating multi-level summarization systems with short, medium, and long-term memory
- Integrating external knowledge into conversation summaries
- Building adaptive systems that learn user preferences over time
- Managing conversation memory efficiently with strategic message removal

Key takeaways:
- **Conversation Summarization**: Essential for managing long conversations within token limits
- **Strategic Summarization**: Different approaches work better for different conversation types
- **Multi-Level Memory**: Hierarchical summaries provide better context management
- **Knowledge Integration**: External knowledge enhances summary quality and response relevance
- **Adaptive Systems**: Learning user preferences improves personalization over time
- **Conditional Logic**: Smart triggering of summarization based on conversation characteristics
- **State Management**: Custom state schemas enable sophisticated conversation tracking

These summarization techniques are crucial for building production-ready conversational AI systems that can handle extended interactions while maintaining context and providing relevant, personalized responses.

Next, complete the final exercise with chatbot-external-memory to learn about persistent storage for conversation memory!